# References
- https://www.w3schools.com/sql/sql_intro.asp
- O'REILLY Learning SQL Generate, Manipulate, and Retrieve Data Alan Beaulieu
- O'REILLY Using SQLITE Jay A. Kreibich

Programmes:
- https://sqlitebrowser.org/
- https://www.youtube.com/watch?v=wGqVjwNpBxY  SQLITE en ligne de commandes

In [1]:
# Global affectation

DATABASE_NAME = 'essai.db'
ATELIERS = ['MEP','Wet','PECVD','PVD','Serig','Sinton']
FOLDER = r'C:\Users\franc\PyVenv\PV_DB_management'

In [2]:
def df2sqlite(dataframe, path_db=None, tbl_name="import"):

    '''The function df2sqlite converts a dataframe into a squlite database.
    
    Args:
       dataframe (panda.DataFrame): the dataframe to convert in a data base
       path_db (Path): full pathname of the database
       tbl_name (str): name of the table
    '''
    
    # Standard library imports
    import sqlite3
    
    # 3rd party imports
    import pandas as pd
    

    if path_db is None:  # Connection to the database
        conn = sqlite3.connect(":memory:")
    else:
        conn = sqlite3.connect(path_db)
        
    # Creates a database and a table
    cur = conn.cursor()
    col_str = '"' + '","'.join(dataframe.columns) + '"'
    cur.execute(f"CREATE TABLE IF NOT EXISTS {tbl_name} ({col_str})")
    dataframe.to_sql(tbl_name, conn, if_exists='replace', index = False)

    cur.close()
    conn.close()
    
def create_parsing_db():
    
    '''Convert the list of .csv file of the parsing FOLDER into a databae in which
    each .csv file corresponds to a table.
    
    Args:
       FOLDER (str): full path of de FOLDER parsing or filt_<i>
    '''
    
    import os
    import pandas as pd

    for file in [file for file in os.listdir(FOLDER) if file.endswith('.xlsx')]:
        df = pd.read_excel(os.path.join(FOLDER,file))
        if len(df) > 0:
            df2sqlite(df, path_db=os.path.join(FOLDER,DATABASE_NAME), tbl_name=file.split('.')[0])
        else: 
            print(f'Warning: the file {file} is empty and will not be added to the database')

def query_db(query):
    
    '''Sends a query to the database.
    
    Args:
       query (str): the sql query
       
    Retuns:
       The result of the query as a dataframe.
    '''
    
    # Standard library import
    import os
    import sqlite3
    
    # 3rd party import
    import pandas as pd
    
    conn = sqlite3.connect(os.path.join(FOLDER,DATABASE_NAME))
    cur = conn.cursor()
    
    try:
        cur.execute(query)
        
    except sqlite3.OperationalError as e:
        raise Exception(f'An errors has occured in the query: {e}')
    else:
        result_querry = {i:list(x) for i,x in enumerate(cur.fetchall())}
        df = pd.DataFrame.from_dict(result_querry).T    
    finally:  
        cur.close()
        conn.close()
    
    return df

def check_id_existance(table,PK):
    
    '''Returns True if the key PK allready exits in the table Returns False ortherwise.
    '''
    
    
    # Standard library import
    import os
    import sqlite3
    from string import Template
    
    
    conn = sqlite3.connect(os.path.join(FOLDER,DATABASE_NAME))
    cur = conn.cursor()
    
    template = Template('''SELECT * FROM $table1
                           WHERE  PK='$PK'
                        ''')
    id_exists = False
    try:
        cur.execute(template.substitute({'table1': table,
                                         'PK': PK,}))
    except sqlite3.OperationalError as e:
        raise Exception(f'An errors has occured in the query: {e}')
    else:
        row = cur.fetchone() 
    finally:  
        cur.close()
        conn.close()
    
    if row: 
        id_exists = True 
    else:
        id_exists = False
    return id_exists


def insert_value(table,value):
    
    # Standard library import
    import os
    import sqlite3
    from string import Template
    
    
    conn = sqlite3.connect(os.path.join(FOLDER,DATABASE_NAME))
    cur = conn.cursor()
    
    template = Template('''INSERT INTO $table
                           VALUES $value
                        ''')
    
    cur.execute(template.substitute({'table': table,
                                     'value': value,}))
    
    conn.commit()
    
    cur.close()
    conn.close()
    


def delete_multiple_records(table,idList):
    
    # Standard library imports
    import os
    import sqlite3
    from string import Template
    
    template = Template('''DELETE from $table
                           WHERE PK = ?
                        ''')
    
    try:
        conn = sqlite3.connect(os.path.join(FOLDER,DATABASE_NAME))
        cur = conn.cursor()
        cur.executemany(template.substitute({'table': table,}),idList)
        conn.commit()
        print("Total", cur.rowcount, "Records deleted successfully")
        cur.close()

    except sqlite3.Error as error:
        print("Failed to delete multiple records from sqlite table", error)
    finally:
        if conn:
            conn.close()
            

In [12]:
# Builds 6 fake excel files and a databse with six corresponding tables with N°Demande <= n_demandes

# Standard library imports
import datetime
import itertools
import random
import string
from pathlib import Path
from datetime import date

# 3rd party imports
import pandas as pd



random_string_generator = lambda n:[''.join(random.choice(string.ascii_letters) for x in range(str_size)) for _ in range(n)]
 
str_size = 12
 
n_demandes = 500 # number of different demands
for atelier in ATELIERS:
    dic = {'N°demande':(n_demande := list(itertools.chain(*[[i]*random.randint(1,5) for i in range(n_demandes)]))),
       'Version':[random.randint(0, 2) for _ in range(len(n_demande))],
       'Split':[random.randint(0, 1) for _ in range(len(n_demande))],
       'Date' : [(date.today() + datetime.timedelta(days=random.randint(0, 5))).isoformat() for _ in range(len(n_demande))],
       'P1': np.random.normal(loc = 0.0, scale = 1.0, size = len(n_demande)),
       'P2': np.random.normal(loc = 0.0, scale = 1.0, size = len(n_demande)),  
       'P3': np.random.normal(loc = 0.0, scale = 1.0, size = len(n_demande)),
       'P4': np.random.normal(loc = 0.0, scale = 1.0, size = len(n_demande)),
       'PX': np.random.normal(loc = 0.0, scale = 1.0, size = len(n_demande)),
       'Commentaire': random_string_generator(len(n_demande))}
    
    dg = pd.DataFrame.from_dict(dic)
    dg = dg.drop_duplicates(['N°demande','Version','Split'])
    dg['PK'] = dg.apply(lambda row:f'{row[0]}_{row[1]}_{row[2]}',axis=1)
    dg.to_excel(Path(FOLDER) / Path(atelier+'.xlsx'),index=False)
    
create_parsing_db()

In [ ]:
# Check if if the primary key already exists

PK = '2_1_0'
for atelier in ATELIERS:
    print(atelier,check_id_existance(atelier,PK))

In [ ]:
# Insert a new value in a table in and only if the primary key doesn't exist

atelier = 'MEP'

dic = {'N°demande':101,
       'Version':1,
       'Split':1,
       'Date' : ( date.today() + datetime.timedelta(days=random.randint(0, 5))).isoformat(),
       'P1': 0.1,
       'P2': 0.1,  
       'P3': 0.5,
       'P4': 0.6,
       'PX': 0.7,
       }
dic['PK'] = f"{dic['N°demande']}_{dic['Version']}_{dic['Split']}"

if not check_id_existance(atelier,dic['PK']):

    insert_value(atelier,tuple(dic.values()))
    

In [ ]:
# Delete records from a table

atelier = 'MEP'
ids_to_delete = [('101_1_1',), ('0_1_1',), ('0_2_1',)]
delete_multiple_records(atelier,ids_to_delete)

In [18]:
# Select values
# SELECT PECVD.PK, Sinton.PK, Sinton.P1 FROM PECVD  LEFT JOIN Sinton ON PECVD.PK=Sinton.PK
#
#CREATE VIEW SYNTHESIS AS 
#SELECT PK,P1,P2 FROM  PECVD;

from pathlib import Path

query = '''SELECT 
              Sinton.P1 AS SINTON_P1,
              MEP.P1 AS MEP_P1,
              PECVD.P1 AS PECVD_P1,
              Wet.P2 AS Wet_P2,
              MEP.PK AS MEP_PEK,
              Sinton.PK AS Sinton_PK,
              PECVD.PK AS PECVD_PK,
              Wet.PK AS Wet_PK
           FROM 
              Sinton 
           JOIN MEP 
              ON Sinton.PK=MEP.PK 
           JOIN PECVD 
              ON Sinton.PK=PECVD.PK 
           JOIN Wet 
              ON Sinton.PK=Wet.PK
           WHERE
              Sinton.P1>0.5 AND Wet.P2>0.5
        '''

script = 'demo_tri.sql'
sql_file = Path(FOLDER) / Path(script)
with open(sql_file,'r') as file:
    query = ''.join(file.readlines())

df = query_db(query)
df.columns = ['SINTON_P1',' MEP_P1','PECVD_P1','Wet_P2','MEP_PEK','Sinton_PK','PECVD_PK','Wet_PK']
df

,SINTON_P1,MEP_P1,PECVD_P1,Wet_P2,MEP_PEK,Sinton_PK,PECVD_PK,Wet_PK
0,1.794713,-1.027422,-1.400332,1.182185,83_0_0,83_0_0,83_0_0,83_0_0
1,1.123293,0.250772,0.428484,0.552821,116_0_1,116_0_1,116_0_1,116_0_1
2,0.615083,0.022209,1.101837,1.000308,162_1_1,162_1_1,162_1_1,162_1_1
3,1.194385,0.23165,-0.058578,1.904245,295_2_0,295_2_0,295_2_0,295_2_0
4,0.615701,-0.784428,1.059565,0.634467,295_1_1,295_1_1,295_1_1,295_1_1
5,1.728823,-0.202104,0.574472,0.822755,308_0_0,308_0_0,308_0_0,308_0_0
6,1.299608,2.283395,-0.571412,0.547502,345_1_0,345_1_0,345_1_0,345_1_0
7,0.52515,0.261233,-0.088461,0.821375,482_1_0,482_1_0,482_1_0,482_1_0


In [ ]:
def decorator(n):
    def decorator_arguments(function):
        def wrapper(*args):
            for x in args:
                print(f" Arguments accepted are: {n*x}")
            return function(*args)

        return wrapper
    return decorator_arguments
  
@decorator(3)
def add(a,b,d):
    return a+b+d
  
def main():
  
    # the reference of wrapper_arguments
    # is returned
    add1 = add
    print(add1)
  
    # passing the arguments to the 
    # wrapper_arguments
    c = add(2,6,7)
    print(c)
  
main()

In [ ]:

def decorator_arguments(function):
    def wrapper(*args):
        for x in args:
            print(f" Arguments accepted are: {n*x}")
        return function(*args)

    return wrapper
    
  
@decorator_arguments
def add(a,b,d):
    return a+b+d
  
def main():
  
    # the reference of wrapper_arguments
    # is returned
    add1 = add
    print(add1)
  
    # passing the arguments to the 
    # wrapper_arguments
    c = add(2,6,7)
    print(c)
  
main()